# Performance dataframes

In [12]:
import os
import re
import json
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
from natsort import index_natsorted

## General parameters

In [59]:
nbr_sims = 5

instance_key = 'instance'
best_obj_key = 'best obj'
worst_obj_key = 'worst obj'
avg_obj_key = 'obj'
max_time_key = 'max time'
min_time_key = 'min time'
avg_time_key = 'time'
max_iter_key = 'max iter'
min_iter_key = 'min iter'
avg_iter_key = 'iter'
set_part_key = 'sp improv'

project_path = os.path.dirname(os.path.abspath('.'))
directory_path = '/output/solstorm/alns/performance/'

generate_df = True

## Functions

In [60]:
def map_instance_to_data(run_path):
    instance_to_data = {}
    for file_name in os.listdir(run_path):
        split_name = re.split('_|\.', file_name)
        instance_name = split_name[0]
        is_history = split_name[2] == 'history'
        if is_history:
            with open(run_path + file_name) as file:
                history_json = json.load(file)
        
            obj = history_json['best_objective']
            time = history_json['runtime']
            it = history_json['number_of_iterations']
            set_part_improv = history_json['number_of_improvements_by_set_partitioning']

            if instance_name in instance_to_data:
                data = instance_to_data[instance_name]

                if obj < data[0]:
                    data[0] = obj
                if obj > data[1]:
                    data[1] = obj
                if time > data[3]:
                    data[3] = time
                if time < data[4]:
                    data[4] = time
                if it > data[6]:
                    data[6] = it
                if it < data[7]:
                    data[7] = it

                data[2] += obj
                data[5] += time
                data[8] += it
                data[9] += set_part_improv
                data[10] += 1
            else:
                instance_to_data[instance_name] = [obj, obj, obj, time, time, time, it, it, it, set_part_improv, 1]
    return instance_to_data

def generate_run_df(run_name):
    run_path = project_path + directory_path + run_name
    
    instance_to_data = map_instance_to_data(run_path)
    
    df = pd.DataFrame(columns=[instance_key, 
                               best_obj_key, worst_obj_key, avg_obj_key, 
                               max_time_key, min_time_key, avg_time_key, 
                               max_iter_key, min_iter_key, avg_iter_key])
    
    for instance in instance_to_data:
        data = instance_to_data[instance]
        best_objective = data[0]
        worst_objective = data[1]
        avg_objective = data[2] / nbr_sims
        max_time = data[3]
        min_time = data[4]
        avg_time = data[5] / nbr_sims
        max_iter = data[6]
        min_iter = data[7]
        avg_iter = data[8] / nbr_sims
        avg_set_part_improv = data[9] / nbr_sims
        
        if data[10] != nbr_sims:
            print(f'{instance} HAS LESS THAN FIVE SIMULATIONS!')
        
        row = pd.Series({instance_key: instance, 
                         best_obj_key: best_objective,
                         worst_obj_key: worst_objective,
                         avg_obj_key: avg_objective, 
                         max_time_key: max_time,
                         min_time_key: min_time,
                         avg_time_key: avg_time, 
                         max_iter_key: max_iter,
                         min_iter_key: min_iter,
                         avg_iter_key: avg_iter,
                         set_part_key: avg_set_part_improv})
        
        df = df.append(row, ignore_index=True)
    
    df = df.sort_values(by='instance',
                        key=lambda x: np.argsort(index_natsorted(df['instance'])),
                        inplace=False)
    df = df.reset_index(drop=True)
    
    mean_row = pd.Series({instance_key: 'Mean values',
                          best_obj_key: df[best_obj_key].mean(),
                          worst_obj_key: df[worst_obj_key].mean(),
                          avg_obj_key: df[avg_obj_key].mean(),
                          max_time_key: df[max_time_key].mean(),
                          min_time_key: df[min_time_key].mean(),
                          avg_time_key: df[avg_time_key].mean(),
                          max_iter_key: df[max_iter_key].mean(),
                          min_iter_key: df[min_iter_key].mean(),
                          avg_iter_key: df[avg_iter_key].mean(),
                          set_part_key: df[set_part_key].mean()})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(1)
    return df

def merge_dfs(dfs):
    df_copies = [df.copy() for df in dfs]
    for df in df_copies:
        df.drop([best_obj_key, worst_obj_key, max_time_key, min_time_key, max_iter_key, min_iter_key], 
                axis=1, inplace=True)
    df_total = pd.concat(df_copies, axis=1)
    
    # Drop duplicate instance columns
    li = [i for i in range(4, len(df_total.columns), 4)]
    df_total = df_total.iloc[:, [j for j, c in enumerate(df_total.columns) if j not in li]]
    
    df_total = df_total.round(1)
    return df_total

def aggregate_df_by_instance_group(df):
    instance_size_to_data = {}
    for idx, row in df.iterrows():
        instance_name = row[instance_key]

        if instance_name == 'Mean values':
            continue
        
        split_name = re.split('-', instance_name)
        instance_size = split_name[0]
        
        best_obj = row[best_obj_key]
        worst_obj = row[worst_obj_key]
        avg_obj = row[avg_obj_key]
        max_time = row[max_time_key]
        min_time = row[min_time_key]
        avg_time = row[avg_time_key]
        max_iter = row[max_iter_key]
        min_iter = row[min_iter_key]
        avg_iter = row[avg_iter_key]
        avg_set_part_improv = row[set_part_key]
        
        if instance_size in instance_size_to_data:
            data = instance_size_to_data[instance_size]
            data[0] += best_obj
            data[1] += worst_obj
            data[2] += avg_obj
            data[3] += max_time
            data[4] += min_time
            data[5] += avg_time
            data[6] += max_iter
            data[7] += min_iter
            data[8] += avg_iter
            data[9] += avg_set_part_improv
            data[10] += 1  # Number of times encountered instance size
        else:
            data = [best_obj, worst_obj, avg_obj, max_time, min_time, avg_time, max_iter, min_iter, avg_iter, avg_set_part_improv, 1]
            instance_size_to_data[instance_size] = data

    df = pd.DataFrame(columns=['instance_group', best_obj_key, worst_obj_key, avg_obj_key, max_time_key, min_time_key,
                               avg_time_key, max_iter_key, min_iter_key, avg_iter_key, set_part_key])
    
    for instance_size in instance_size_to_data:
        data = instance_size_to_data[instance_size]
        nbr_sims = data[10]
        if nbr_sims < 5:
            print(f'{instance_size} INSTANCE_SIZE LESS THAN FIVE INSTANCES!')
        
        row = pd.Series({'instance_group': instance_size, 
                         best_obj_key: data[0] / nbr_sims,
                         worst_obj_key: data[1] / nbr_sims,
                         avg_obj_key: data[2] / nbr_sims, 
                         max_time_key: data[3] / nbr_sims,
                         min_time_key: data[4] / nbr_sims,
                         avg_time_key: data[5] / nbr_sims, 
                         max_iter_key: data[6] / nbr_sims,
                         min_iter_key: data[7] / nbr_sims,
                         avg_iter_key: data[8] / nbr_sims,
                         set_part_key: data[9] / nbr_sims})
    
        df = df.append(row, ignore_index=True)
        
    mean_row = pd.Series({'instance_group': 'Mean values',
                          best_obj_key: df[best_obj_key].mean(),
                          worst_obj_key: df[worst_obj_key].mean(),
                          avg_obj_key: df[avg_obj_key].mean(),
                          max_time_key: df[max_time_key].mean(),
                          min_time_key: df[min_time_key].mean(),
                          avg_time_key: df[avg_time_key].mean(),
                          max_iter_key: df[max_iter_key].mean(),
                          min_iter_key: df[min_iter_key].mean(),
                          avg_iter_key: df[avg_iter_key].mean(),
                          set_part_key: df[set_part_key].mean()})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(1)
    return df

def load_df(file_name):
    run_df = pd.read_pickle(f'dataframes/performance/{file_name}')
    # run_df = sort_df(run_df, sort_column)
    return run_df

## ALNS baseline

In [61]:
if generate_df:
    run_baseline_name = 'second/baseline/'
    run_baseline_df = generate_run_df(run_baseline_name)
    run_baseline_agg_df = aggregate_df_by_instance_group(run_baseline_df)

    run_baseline_file_name = f'dataframes/performance/baseline.pkl'
    run_baseline_agg_file_name = f'dataframes/performance/baseline_agg.pkl'
    run_baseline_df.to_pickle(run_baseline_file_name)
    run_baseline_agg_df.to_pickle(run_baseline_agg_file_name)

In [62]:
run_baseline_df = load_df('baseline.pkl')
run_baseline_df

,instance,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5-5-1-1,2519.9,2519.9,2519.9,0.7,0.7,0.7,5000,5000,5000.0,0.0
1,5-5-1-2,2214.2,2214.2,2214.2,0.7,0.7,0.7,5000,5000,5000.0,0.0
2,5-6-1-1,1939.6,1939.6,1939.6,0.9,0.8,0.9,5000,5000,5000.0,0.0
3,5-7-1-1,2709.7,2709.7,2709.7,1.3,1.3,1.3,5000,5000,5000.0,0.0
4,5-7-1-2,1704.6,1704.6,1704.6,1.1,1.1,1.1,5000,5000,5000.0,0.0
5,7-8-1-1,2046.9,2046.9,2046.9,1.9,1.7,1.8,5000,5000,5000.0,0.0
6,7-8-1-2,1971.3,1971.3,1971.3,1.8,1.6,1.7,5000,5000,5000.0,0.0
7,7-8-1-3,2071.9,2071.9,2071.9,2.0,1.8,1.9,5000,5000,5000.0,0.0
8,7-9-1-1,2170.4,2170.4,2170.4,4.6,4.1,4.3,5000,5000,5000.0,0.0
9,7-9-1-2,2206.1,2206.1,2206.1,2.8,2.0,2.4,5000,5000,5000.0,0.0


In [63]:
run_baseline_agg_df = load_df('baseline_agg.pkl')
run_baseline_agg_df

,instance_group,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5,2217.6,2217.6,2217.6,0.9,0.9,0.9,5000.0,5000.0,5000.0,0.0
1,7,2093.3,2093.3,2093.3,2.6,2.2,2.4,5000.0,5000.0,5000.0,0.0
2,9,5627.5,5627.5,5627.5,4.9,4.1,4.5,5000.0,5000.0,5000.0,0.0
3,11,3517.5,3517.5,3517.5,19.9,17.7,18.8,5000.0,5000.0,5000.0,0.0
4,13,3970.5,3984.8,3973.4,33.6,30.0,32.0,5000.0,5000.0,5000.0,0.0
5,15,8370.3,8446.1,8402.1,40.7,33.4,37.0,5000.0,5000.0,5000.0,0.0
6,17,4762.1,4981.4,4927.4,61.8,51.0,56.9,5000.0,5000.0,5000.0,0.0
7,19,4936.4,5171.4,5110.6,78.5,64.5,72.8,5000.0,5000.0,5000.0,0.0
8,21,9274.7,9338.0,9313.5,89.4,77.0,82.1,5000.0,5000.0,5000.0,0.0
9,23,5676.3,6004.3,5903.8,144.4,122.7,134.6,5000.0,5000.0,5000.0,0.0


## Sequential ALNS

In [44]:
if generate_df:
    run_sequential_name = 'first/sequential/'
    run_sequential_df = generate_run_df(run_sequential_name)
    run_sequential_agg_df = aggregate_df_by_instance_group(run_sequential_df)

    run_sequential_file_name = f'dataframes/performance/sequential.pkl'
    run_sequential_agg_file_name = f'dataframes/performance/sequential_agg.pkl'
    run_sequential_df.to_pickle(run_sequential_file_name)
    run_sequential_agg_df.to_pickle(run_sequential_agg_file_name)

In [45]:
run_sequential_df = load_df('sequential.pkl')
run_sequential_df

,instance,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5-5-1-1,2519.9,2519.9,2519.9,0.7,0.7,0.7,5000,5000,5000.0,0.0
1,5-5-1-2,2214.2,2214.2,2214.2,0.7,0.7,0.7,5000,5000,5000.0,0.0
2,5-6-1-1,1939.6,1939.6,1939.6,0.8,0.8,0.8,5000,5000,5000.0,0.0
3,5-7-1-1,2709.7,2709.7,2709.7,1.4,1.3,1.3,5000,5000,5000.0,0.0
4,5-7-1-2,1704.6,1704.6,1704.6,1.2,1.2,1.2,5000,5000,5000.0,0.0
5,7-8-1-1,2046.9,2046.9,2046.9,2.2,1.9,2.1,5000,5000,5000.0,0.0
6,7-8-1-2,1971.3,1971.3,1971.3,5.6,5.4,5.5,5000,5000,5000.0,0.0
7,7-8-1-3,2071.9,2071.9,2071.9,3.6,3.4,3.5,5000,5000,5000.0,0.0
8,7-9-1-1,2170.4,2170.4,2170.4,4.8,4.7,4.8,5000,5000,5000.0,0.0
9,7-9-1-2,2206.1,2206.1,2206.1,6.7,6.3,6.6,5000,5000,5000.0,0.0


In [46]:
run_sequential_agg_df = load_df('sequential_agg.pkl')
run_sequential_agg_df

,instance_group,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5,2217.6,2217.6,2217.6,1.0,0.9,0.9,5000.0,5000.0,5000.0,0.0
1,7,2093.3,2093.3,2093.3,4.6,4.3,4.5,5000.0,5000.0,5000.0,0.0
2,9,5627.5,5627.5,5627.5,10.8,9.5,10.1,5000.0,5000.0,5000.0,0.0
3,11,3517.5,3517.5,3517.5,29.9,26.0,28.0,5000.0,5000.0,5000.0,0.0
4,13,3970.5,3970.5,3970.5,67.4,56.4,61.7,5000.0,5000.0,5000.0,0.0
5,15,6179.0,8399.9,7944.4,86.7,74.9,80.7,5000.0,5000.0,5000.0,0.0
6,17,4827.9,4986.8,4943.2,151.6,124.0,138.8,5000.0,5000.0,5000.0,0.0
7,19,4926.9,5173.1,5095.8,190.6,166.2,178.9,5000.0,5000.0,5000.0,0.0
8,21,8362.7,9332.5,9116.9,238.3,201.3,218.0,5000.0,5000.0,5000.0,0.0
9,23,5619.4,5995.6,5871.9,361.2,326.2,342.6,5000.0,5000.0,5000.0,0.0


## LNS

In [19]:
if generate_df:
    run_lns_name = 'second/lns/'
    run_lns_df = generate_run_df(run_lns_name)
    run_lns_agg_df = aggregate_df_by_instance_group(run_lns_df)

    run_lns_file_name = f'dataframes/performance/lns.pkl'
    run_lns_agg_file_name = f'dataframes/performance/lns_agg.pkl'
    run_lns_df.to_pickle(run_lns_file_name)
    run_lns_agg_df.to_pickle(run_lns_agg_file_name)

5-5-1-2 HAS LESS THAN FIVE SIMULATIONS!


In [47]:
run_lns_df = load_df('lns.pkl')
run_lns_df

,instance,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5-5-1-1,2519.9,2519.9,2519.9,0.7,0.7,0.7,5000,5000,5000.0,0.0
1,5-5-1-2,2214.2,2214.2,1771.4,0.7,0.7,0.5,5000,5000,4000.0,0.0
2,5-6-1-1,1939.6,1939.6,1939.6,0.9,0.8,0.8,5000,5000,5000.0,0.0
3,5-7-1-1,2709.7,2709.7,2709.7,1.3,1.3,1.3,5000,5000,5000.0,0.0
4,5-7-1-2,1704.6,1704.6,1704.6,1.1,1.1,1.1,5000,5000,5000.0,0.0
5,7-8-1-1,2046.9,2046.9,2046.9,2.0,1.7,1.8,5000,5000,5000.0,0.0
6,7-8-1-2,1971.3,1971.3,1971.3,1.7,1.5,1.6,5000,5000,5000.0,0.0
7,7-8-1-3,2071.9,2071.9,2071.9,2.0,1.8,1.9,5000,5000,5000.0,0.0
8,7-9-1-1,2170.4,2170.4,2170.4,4.5,4.2,4.3,5000,5000,5000.0,0.0
9,7-9-1-2,2206.1,2206.1,2206.1,2.8,2.1,2.4,5000,5000,5000.0,0.0


In [48]:
run_lns_agg_df = load_df('lns_agg.pkl')
run_lns_agg_df

,instance_group,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5,2217.6,2217.6,2129.0,0.9,0.9,0.9,5000.0,5000.0,4800.0,0.0
1,7,2093.3,2093.3,2093.3,2.6,2.3,2.4,5000.0,5000.0,5000.0,0.0
2,9,5627.5,5627.5,5627.5,4.9,4.2,4.4,5000.0,5000.0,5000.0,0.0
3,11,3517.5,3517.5,3517.5,19.5,16.8,18.1,5000.0,5000.0,5000.0,0.0
4,13,3876.8,3970.5,3951.8,34.0,31.2,32.7,5000.0,5000.0,5000.0,0.0
5,15,8370.3,8452.8,8404.4,39.0,34.2,36.5,5000.0,5000.0,5000.0,0.0
6,17,4930.2,4982.1,4966.0,62.1,53.7,57.1,5000.0,5000.0,5000.0,0.0
7,19,5043.7,5167.3,5130.4,75.2,66.0,70.4,5000.0,5000.0,5000.0,0.0
8,21,8292.2,9371.2,9115.1,90.3,74.4,82.3,5000.0,5000.0,5000.0,0.0
9,23,5806.4,6012.2,5931.2,135.5,115.3,126.9,5000.0,5000.0,5000.0,0.0


## ALNS + local search

In [49]:
if generate_df:
    run_ls_name = 'second/ls/'
    run_ls_df = generate_run_df(run_ls_name)
    run_ls_agg_df = aggregate_df_by_instance_group(run_ls_df)

    run_ls_file_name = f'dataframes/performance/ls.pkl'
    run_ls_agg_file_name = f'dataframes/performance/ls_agg.pkl'
    run_ls_df.to_pickle(run_ls_file_name)
    run_ls_agg_df.to_pickle(run_ls_agg_file_name)

In [50]:
run_ls_df = load_df('ls.pkl')
run_ls_df

,instance,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5-5-1-1,2519.9,2519.9,2519.9,1.5,1.4,1.4,5000,5000,5000.0,0.0
1,5-5-1-2,2214.2,2214.2,2214.2,1.4,1.3,1.3,5000,5000,5000.0,0.0
2,5-6-1-1,1939.6,1939.6,1939.6,1.5,1.5,1.5,5000,5000,5000.0,0.0
3,5-7-1-1,2709.7,2709.7,2709.7,2.2,2.1,2.1,5000,5000,5000.0,0.0
4,5-7-1-2,1704.6,1704.6,1704.6,1.8,1.7,1.8,5000,5000,5000.0,0.0
5,7-8-1-1,2046.9,2046.9,2046.9,4.2,3.8,4.1,5000,5000,5000.0,0.0
6,7-8-1-2,1971.3,1971.3,1971.3,4.4,3.4,4.0,5000,5000,5000.0,0.0
7,7-8-1-3,2071.9,2071.9,2071.9,4.7,4.4,4.5,5000,5000,5000.0,0.0
8,7-9-1-1,2170.4,2170.4,2170.4,7.3,6.8,7.0,5000,5000,5000.0,0.0
9,7-9-1-2,2206.1,2206.1,2206.1,5.4,4.9,5.1,5000,5000,5000.0,0.0


In [51]:
run_ls_agg_df = load_df('ls_agg.pkl')
run_ls_agg_df

,instance_group,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5,2217.6,2217.6,2217.6,1.7,1.6,1.6,5000.0,5000.0,5000.0,0.0
1,7,2093.3,2093.3,2093.3,5.2,4.7,4.9,5000.0,5000.0,5000.0,0.0
2,9,5627.5,5627.5,5627.5,9.2,8.1,8.7,5000.0,5000.0,5000.0,0.0
3,11,3301.9,3517.5,3305.6,53.4,42.4,46.3,5000.0,5000.0,4800.0,0.0
4,13,3543.3,3970.5,3792.9,94.0,75.3,87.1,5000.0,5000.0,5000.0,0.0
5,15,6175.5,8389.5,7532.6,96.2,66.4,82.8,5000.0,5000.0,5000.0,0.0
6,17,4548.2,4946.9,4797.0,195.2,131.7,168.8,5000.0,5000.0,5000.0,0.0
7,19,4693.7,5140.3,4742.4,264.7,193.2,216.5,5000.0,5000.0,4800.0,0.0
8,21,7029.0,9318.7,8283.1,205.8,147.5,183.6,5000.0,5000.0,5000.0,0.0
9,23,5074.5,5814.8,5538.3,445.8,271.8,377.5,5000.0,4774.4,4934.0,0.0


## ALNS + set partitioning

In [52]:
if generate_df: 
    run_sp_name = 'second/sp/'
    run_sp_df = generate_run_df(run_sp_name)
    run_sp_agg_df = aggregate_df_by_instance_group(run_sp_df)

    run_sp_file_name = f'dataframes/performance/sp.pkl'
    run_sp_agg_file_name = f'dataframes/performance/sp_agg.pkl'
    run_sp_df.to_pickle(run_sp_file_name)
    run_sp_agg_df.to_pickle(run_sp_agg_file_name)

In [54]:
run_sp_df = load_df('sp.pkl')
run_sp_df

,instance,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5-5-1-1,2519.9,2519.9,2519.9,0.8,0.8,0.8,5000,5000,5000.0,0.0
1,5-5-1-2,2214.2,2214.2,2214.2,0.8,0.7,0.8,5000,5000,5000.0,0.0
2,5-6-1-1,1939.6,1939.6,1939.6,0.9,0.9,0.9,5000,5000,5000.0,0.0
3,5-7-1-1,2709.7,2709.7,2709.7,1.4,1.3,1.4,5000,5000,5000.0,0.0
4,5-7-1-2,1704.6,1704.6,1704.6,1.3,1.2,1.2,5000,5000,5000.0,0.0
5,7-8-1-1,2046.9,2046.9,2046.9,2.0,1.8,2.0,5000,5000,5000.0,0.0
6,7-8-1-2,1971.3,1971.3,1971.3,2.0,1.7,1.8,5000,5000,5000.0,0.0
7,7-8-1-3,2071.9,2071.9,2071.9,2.2,2.0,2.1,5000,5000,5000.0,0.0
8,7-9-1-1,2170.4,2170.4,2170.4,4.7,4.5,4.6,5000,5000,5000.0,0.0
9,7-9-1-2,2206.1,2206.1,2206.1,3.0,2.2,2.4,5000,5000,5000.0,0.0


In [55]:
run_sp_agg_df = load_df('sp_agg.pkl')
run_sp_agg_df

,instance_group,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5,2217.6,2217.6,2217.6,1.0,1.0,1.0,5000.0,5000.0,5000.0,0.0
1,7,2093.3,2093.3,2093.3,2.8,2.4,2.6,5000.0,5000.0,5000.0,0.0
2,9,5627.5,5627.5,5550.2,5.5,4.6,4.6,5000.0,5000.0,4800.0,0.0
3,11,3517.5,3517.5,3517.5,23.3,20.0,21.4,5000.0,5000.0,5000.0,0.0
4,13,3871.0,3974.6,3935.9,39.6,33.8,37.1,5000.0,5000.0,5000.0,0.0
5,15,8374.5,8399.4,8383.9,45.1,39.1,42.1,5000.0,5000.0,5000.0,0.8
6,17,4893.0,5008.7,4965.9,75.3,61.7,68.8,5000.0,5000.0,5000.0,1.2
7,19,4866.2,5160.5,5084.7,100.3,80.2,91.6,5000.0,5000.0,5000.0,1.9
8,21,7778.2,9323.0,8677.8,121.6,103.6,110.1,5000.0,5000.0,5000.0,2.4
9,23,5698.0,5965.1,5870.1,186.0,150.8,166.6,5000.0,5000.0,5000.0,2.8


## ALNS + local search + set partitioning

In [56]:
if generate_df:
    run_lssp_name = 'second/lssp/'
    run_lssp_df = generate_run_df(run_lssp_name)
    run_lssp_agg_df = aggregate_df_by_instance_group(run_lssp_df)

    run_lssp_file_name = f'dataframes/performance/lssp.pkl'
    run_lssp_agg_file_name = f'dataframes/performance/lssp_agg.pkl'
    run_lssp_df.to_pickle(run_lssp_file_name)
    run_lssp_agg_df.to_pickle(run_lssp_agg_file_name)

In [57]:
run_ls_sp_df = load_df('lssp.pkl')
run_ls_sp_df

FileNotFoundError: [Errno 2] No such file or directory: 'dataframes/performance/lssp.pkl'

In [58]:
run_ls_sp_agg_df = load_df('lssp_agg.pkl')
run_ls_sp_agg_df

FileNotFoundError: [Errno 2] No such file or directory: 'dataframes/performance/lssp_agg.pkl'

## Parallel vs. sequential heuristics

In [229]:
baseline_sequential_df = merge_dfs([run_baseline_df, run_sequential_df])
baseline_sequential_agg_df = merge_dfs([run_baseline_agg_df, run_sequential_agg_df])

In [230]:
baseline_sequential_df

,instance,obj,time,iter,obj,time,iter
0,5-5-1-1,2519.9,0.7,5000.0,2519.9,0.7,5000.0
1,5-5-1-2,2214.2,0.7,5000.0,2214.2,0.7,5000.0
2,5-6-1-1,1939.6,0.9,5000.0,1939.6,0.8,5000.0
3,5-7-1-1,2709.7,1.4,5000.0,2709.7,1.3,5000.0
4,5-7-1-2,1704.6,1.2,5000.0,1704.6,1.2,5000.0
5,7-8-1-1,2046.9,1.9,5000.0,2046.9,2.1,5000.0
6,7-8-1-2,1971.3,4.6,5000.0,1971.3,5.5,5000.0
7,7-8-1-3,2071.9,3.4,5000.0,2071.9,3.5,5000.0
8,7-9-1-1,2170.4,4.1,5000.0,2170.4,4.8,5000.0
9,7-9-1-2,2206.1,5.6,5000.0,2206.1,6.6,5000.0


In [231]:
baseline_sequential_agg_df

,instance_group,obj,time,iter,obj,time,iter
0,5,2217.6,1.0,5000.0,2217.6,0.9,5000.0
1,7,2093.3,3.9,5000.0,2093.3,4.5,5000.0
2,9,5617.6,7.8,5000.0,5627.5,10.1,5000.0
3,11,3449.7,24.0,5000.0,3517.5,28.0,5000.0
4,13,3865.5,50.0,5000.0,3970.5,61.7,5000.0
5,15,7436.0,60.6,5000.0,7944.4,80.7,5000.0
6,17,4920.4,105.2,5000.0,4943.2,138.8,5000.0
7,19,5050.4,130.3,5000.0,5095.8,178.9,5000.0
8,21,8628.6,161.2,5000.0,9116.9,218.0,5000.0
9,23,5886.4,235.5,5000.0,5871.9,342.6,5000.0


## ALNS vs. LNS

In [232]:
baseline_lns_df = merge_dfs([run_baseline_df, run_lns_df])
baseline_lns_df

,instance,obj,time,iter,obj,time,iter
0,5-5-1-1,2519.9,0.7,5000.0,2519.9,0.7,5000.0
1,5-5-1-2,2214.2,0.7,5000.0,2214.2,0.7,5000.0
2,5-6-1-1,1939.6,0.9,5000.0,1939.6,0.8,5000.0
3,5-7-1-1,2709.7,1.4,5000.0,2709.7,1.4,5000.0
4,5-7-1-2,1704.6,1.2,5000.0,1363.7,0.9,4000.0
5,7-8-1-1,2046.9,1.9,5000.0,2046.9,1.9,5000.0
6,7-8-1-2,1971.3,4.6,5000.0,1971.3,4.6,5000.0
7,7-8-1-3,2071.9,3.4,5000.0,2071.9,3.4,5000.0
8,7-9-1-1,2170.4,4.1,5000.0,2170.4,4.0,5000.0
9,7-9-1-2,2206.1,5.6,5000.0,2206.1,5.6,5000.0


## ALNS vs. ALNS + local search

In [174]:
baseline_ls_df = merge_dfs([run_baseline_df, run_ls_df])
baseline_ls_df

17-22-3-1 HAS LESS THAN FIVE SIMULATIONS!


## ALNS vs. ALNS + set partitioning

## ALNS vs. ALNS + local search + set partitioning